In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import torch

In [2]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [3]:
df = pd.read_csv('data/preprocessed_data_stemmed.csv', delimiter=';')
df

,Unnamed: 0,id,country,name,NACE,description,text,text_no_stopwords,description_no_stopwords,text_stemmed,description_stemmed
0,0,SE5560330788,SE,Geberit Production AB,23420.0,the globally operating geberit group is a euro...,the globally operating geberit group is a euro...,globally operating geberit group european lead...,globally operating geberit group european lead...,global oper geberit group european leader fiel...,global oper geberit group european leader fiel...
1,1,SE5569037251,SE,UMAB AB,71120.0,company should provide services within mechani...,company should provide services within mechani...,company provide services within mechanical pro...,company provide services within mechanical pro...,compani provid servic within mechan project in...,compani provid servic within mechan project in...
2,2,SE5592295116,SE,Proarch AB,71110.0,sell consulting services within real estate ar...,sell consulting services within real estate ar...,sell consulting services within real estate ar...,sell consulting services within real estate ar...,sell consult servic within real estat architec...,sell consult servic within real estat architec...
3,3,SE5590218375,SE,Double Duck AB,90030.0,aktiebolaget should conduct authoring activiti...,aktiebolaget should conduct authoring activiti...,aktiebolaget conduct authoring activities ther...,aktiebolaget conduct authoring activities ther...,aktiebolaget conduct author activ therebi comp...,aktiebolaget conduct author activ therebi comp...
4,4,DK32305148,DK,RØRVIG FISK'S FISKERESTAURANT ApS,56100.0,company purpose is to drive restaurant and sec...,company purpose is to drive restaurant and sec...,company purpose drive restaurant second manage...,company purpose drive restaurant second manage...,compani purpos drive restaur second manag perc...,compani purpos drive restaur second manag perc...
...,...,...,...,...,...,...,...,...,...,...,...
74124,82113,DK29616450,DK,HOWDEN HOLDINGS ApS,64200.0,manufacturer of fans heaters compressors blowe...,manufacturer of fans heaters compressors blowe...,manufacturer fans heaters compressors blowers ...,manufacturer fans heaters compressors blowers ...,manufactur fan heater compressor blower steam ...,manufactur fan heater compressor blower steam ...
74125,82114,DK32152791,DK,Viking Wind ApS,28110.0,company purpose is development manufacture an...,company purpose is development manufacture an...,company purpose development manufacture sale w...,company purpose development manufacture sale w...,compani purpos develop manufactur sale wind po...,compani purpos develop manufactur sale wind po...
74126,82115,DK60787816,DK,CAMTO A/S,27120.0,camto delivers protection systems to industry ...,camto delivers protection systems to industry ...,camto delivers protection systems industry amo...,camto delivers protection systems industry amo...,camto deliv protect system industri among firs...,camto deliv protect system industri among firs...
74127,82116,DK34726647,DK,SØNDERSØ ENERGI A/S,71120.0,company purpose is sale and mounting of renewa...,company purpose is sale and mounting of renewa...,company purpose sale mounting renewable energy...,company purpose sale mounting renewable energy...,compani purpos sale mount renew energi plant g...,compani purpos sale mount renew energi plant g...


In [4]:
train_df = pd.read_csv('data/train/Computer Vision.csv', delimiter=';')
initials_id = train_df[train_df['AI search'] == 'Initial']['Firmnav ID'].tolist()
positives_id = train_df[(train_df['Rating'] == 1) & (train_df['AI search'] != 'Initial')]['Firmnav ID'].tolist()
negatives_id = train_df[(train_df['Rating'] == 0) & (train_df['AI search'] != 'Initial')]['Firmnav ID'].tolist()

In [5]:
corpus_id = positives_id + negatives_id
print(len(corpus_id))
print(corpus_id)

94
['SE5590053376', 'SE5568401433', 'DK30575679', 'SE5565758371', 'DK36718285', 'SE5567127823', 'FI24214580', 'DK38898701', 'FI27183214', 'NO998927854', 'FI27580682', 'FI22588407', 'FI06620331', 'DK32842070', 'NO992102403', 'FI16457849', 'SE5567054571', 'FI29078581', 'SE5563194041', 'SE5569826844', 'DK27446434', 'SE5565505400', 'DK29973342', 'NO989527460', 'NO914079845', 'FI16201990', 'DK10969042', 'DK29142068', 'SE5590156104', 'FI19403934', 'DK32449891', 'SE5567761811', 'SE5561871012', 'DK33063288', 'NO991711392', 'NO983298664', 'DK25086317', 'FI20734247', 'FI20865474', 'DK26382823', 'DK28845936', 'FI20039498', 'FI06413949', 'FI01006500', 'SE5566767967', 'NO984157789', 'DK20366532', 'DK71243419', 'SE5591214688', 'SE5563361681', 'FI25489009', 'NO916823320', 'DK17990985', 'FI17870723', 'SE5590698865', 'FI23821450', 'SE5568037898', 'DK27587887', 'FI26290819', 'NO977337798', 'NO998695449', 'SE5569387664', 'FI24778474', 'FI22926197', 'SE5566105655', 'SE5590350079', 'NO991137092', 'SE556241

In [6]:
initials_id

['NO996343545',
 'NO988943037',
 'SE5567101133',
 'SE5566286919',
 'DK38973231',
 'DK37504149',
 'DK36910097',
 'FI20821316']

In [7]:
initials_txt = []
positives_txt = []
negatives_txt = []
for fid in initials_id:
    txt = df[df['id'] == fid].text_stemmed.tolist()
    if len(txt) > 0:
        initials_txt.append(txt)
for fid in positives_id:
    txt = df[df['id'] == fid].text_stemmed.tolist()
    if len(txt) > 1:
        txt = [txt[0]]
    if len(txt) > 0:
        positives_txt.append(txt)
for fid in negatives_id:
    txt = df[df['id'] == fid].text_stemmed.tolist()
    if len(txt) > 1:
        txt = [txt[0]]
    if len(txt) > 0:
        negatives_txt.append(txt)

In [8]:
print(len(initials_txt))
print(len(positives_txt))
print(len(negatives_txt))

6
24
48


In [9]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [10]:
qs = initials_txt
qs = [item for sublist in qs for item in sublist]

In [11]:
corpus = positives_txt + negatives_txt
print(len(corpus))
corpus = [item for sublist in corpus for item in sublist]
len(corpus)

72


72

In [12]:
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [13]:
from sentence_transformers import util

In [14]:
top_k = 5
for i, query in enumerate(qs):
    query_embedding = embedder.encode(query, convert_to_tensor=True)


    # We use cosine-similarity and torch.topk to find the highest 5 scores
    '''cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    #print("Query:", query)
    print('ID', initials_id[i])
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print(corpus_id[idx], "(Score: {:.4f})".format(score))'''

    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    print('ID', initials_id[i])
    for hit in hits:
        print(corpus_id[hit['corpus_id']], "(Score: {:.5f})".format(hit['score']))
    

ID NO996343545
NO914079845 (Score: 0.85546)
SE5590156104 (Score: 0.55316)
SE5567127823 (Score: 0.53524)
DK32449891 (Score: 0.52216)
SE5569826844 (Score: 0.52019)
ID NO988943037
SE5563194041 (Score: 0.83895)
NO992102403 (Score: 0.79902)
SE5591214688 (Score: 0.78867)
DK30575679 (Score: 0.78458)
SE5568037898 (Score: 0.78193)
ID SE5567101133
FI24214580 (Score: 0.75007)
FI22588407 (Score: 0.73708)
DK32842070 (Score: 0.72377)
NO977337798 (Score: 0.71480)
DK30575679 (Score: 0.71260)
ID SE5566286919
DK33063288 (Score: 0.75733)
SE5590350079 (Score: 0.75537)
SE5590053376 (Score: 0.75248)
SE5567761811 (Score: 0.73615)
FI27183214 (Score: 0.72267)
ID DK38973231
DK33063288 (Score: 0.75733)
SE5590350079 (Score: 0.75537)
SE5590053376 (Score: 0.75248)
SE5567761811 (Score: 0.73615)
FI27183214 (Score: 0.72267)
ID DK37504149
SE5567127823 (Score: 0.84187)
SE5591214688 (Score: 0.77687)
FI16457849 (Score: 0.76241)
NO992102403 (Score: 0.73912)
SE5563194041 (Score: 0.73885)
ID DK36910097
NO992102403 (Score: 0.

In [15]:
trainfiles = ['Computer vision.csv', 'Consulting.csv', 'Fintech.csv', 'Fish processing equipment.csv', 'Healthcare.csv', 
                'House builders.csv', 'Industrial vertical investor.csv', 'Innovative.csv', 'IoT.csv', 'IT freelance.csv', 
                'M&A advisors.csv', 'Manufacturers.csv', 'Online games.csv', 'Payments tech.csv', 'PE fund.csv', 
                'Procurement software.csv', 'Resource-efficiency.csv', 'Sustainability.csv', 'SaaS.csv', 
                'Wind turbine tech.csv', ]
output = {name:{} for name in trainfiles}
output

{'Computer vision.csv': {},
 'Consulting.csv': {},
 'Fintech.csv': {},
 'Fish processing equipment.csv': {},
 'Healthcare.csv': {},
 'House builders.csv': {},
 'Industrial vertical investor.csv': {},
 'Innovative.csv': {},
 'IoT.csv': {},
 'IT freelance.csv': {},
 'M&A advisors.csv': {},
 'Manufacturers.csv': {},
 'Online games.csv': {},
 'Payments tech.csv': {},
 'PE fund.csv': {},
 'Procurement software.csv': {},
 'Resource-efficiency.csv': {},
 'Sustainability.csv': {},
 'SaaS.csv': {},
 'Wind turbine tech.csv': {}}

In [16]:
for file in trainfiles:
    train_df = pd.read_csv(f'data/train/{file}', delimiter=';')
    initials_id = train_df[train_df['AI search'] == 'Initial']['Firmnav ID'].tolist()
    positives_id = train_df[(train_df['Rating'] == 1) & (train_df['AI search'] != 'Initial')]['Firmnav ID'].tolist()
    negatives_id = train_df[(train_df['Rating'] == 0) & (train_df['AI search'] != 'Initial')]['Firmnav ID'].tolist()
    
    corpus_id = positives_id + negatives_id
    
    initials_txt = []
    positives_txt = []
    negatives_txt = []
    for fid in initials_id:
        txt = df[df['id'] == fid].text_stemmed.tolist()
        if len(txt) > 1:
            txt = [txt[0]]
        if len(txt) > 0:
            initials_txt.append(txt)
    for fid in positives_id:
        txt = df[df['id'] == fid].text_stemmed.tolist()
        if len(txt) > 1:
            txt = [txt[0]]
        if len(txt) > 0:
            positives_txt.append(txt)
    for fid in negatives_id:
        txt = df[df['id'] == fid].text_stemmed.tolist()
        if len(txt) > 1:
            txt = [txt[0]]
        if len(txt) > 0:
            negatives_txt.append(txt)
            
    qs = initials_txt
    qs = [item for sublist in qs for item in sublist]
    
    corpus = positives_txt + negatives_txt
    corpus = [item for sublist in corpus for item in sublist]

    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    
    top_k = 5
    score = []
    for i, query in enumerate(qs):
        query_embedding = embedder.encode(query, convert_to_tensor=True)


        # We use cosine-similarity and torch.topk to find the highest 5 scores
        '''cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=top_k)

        print("\n\n======================\n\n")
        #print("Query:", query)
        print('ID', initials_id[i])
        print("\nTop 5 most similar sentences in corpus:")

        for score, idx in zip(top_results[0], top_results[1]):
            print(corpus_id[idx], "(Score: {:.4f})".format(score))'''

        # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
        hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
        hits = hits[0]      #Get the hits for the first query
        #print('ID', initials_id[i])
        for hit in hits:
            #print(corpus_id[hit['corpus_id']], "(Score: {:.5f})".format(hit['score']))
            score.append((corpus_id[hit['corpus_id']], hit['score']))
        if 'top_5' not in output[file].keys():
            output[file]['top_5'] = score
    score = sorted(score, key=lambda x:x[1], reverse=True)
    output[file]['top_5'] = score[:5]

In [17]:
output

{'Computer vision.csv': {'top_5': [('NO914079845', 0.8554579019546509),
   ('SE5567127823', 0.8418716192245483),
   ('SE5563194041', 0.8389506340026855),
   ('NO992102403', 0.7990180253982544),
   ('SE5591214688', 0.7886717319488525)]},
 'Consulting.csv': {'top_5': [('DK36076011', 0.8274497985839844),
   ('DK12516835', 0.8232511281967163),
   ('DK25606965', 0.8203682899475098),
   ('DK30714520', 0.8159401416778564),
   ('SE5562540673', 0.8149929642677307)]},
 'Fintech.csv': {'top_5': [('SE5569622441', 0.904498279094696),
   ('FI01170116', 0.8300429582595825),
   ('NO864936792', 0.8146194219589233),
   ('DK64806815', 0.8021937608718872),
   ('DK15134275', 0.8016077280044556)]},
 'Fish processing equipment.csv': {'top_5': [('FI18091246', 0.702485203742981),
   ('FI18091246', 0.7016414403915405),
   ('DK33786646', 0.6925486326217651),
   ('NO940970172', 0.6925486326217651),
   ('DK79104213', 0.6925486326217651)]},
 'Healthcare.csv': {'top_5': [('SE5567909428', 0.6710702180862427),
   ('SE

In [18]:
out_df = pd.DataFrame.from_dict(output)
out_df

,Computer vision.csv,Consulting.csv,Fintech.csv,Fish processing equipment.csv,Healthcare.csv,House builders.csv,Industrial vertical investor.csv,Innovative.csv,IoT.csv,IT freelance.csv,M&A advisors.csv,Manufacturers.csv,Online games.csv,Payments tech.csv,PE fund.csv,Procurement software.csv,Resource-efficiency.csv,Sustainability.csv,SaaS.csv,Wind turbine tech.csv
top_5,"[(NO914079845, 0.8554579019546509), (SE5567127...","[(DK36076011, 0.8274497985839844), (DK12516835...","[(SE5569622441, 0.904498279094696), (FI0117011...","[(FI18091246, 0.702485203742981), (FI18091246,...","[(SE5567909428, 0.6710702180862427), (SE556894...","[(DK27920691, 0.9202667474746704), (DK18411709...","[(SE5563748309, 0.8689890503883362), (FI168014...","[(FI09899489, 0.7718906998634338), (FI04135690...","[(FI25904137, 1.0), (NO990618976, 0.9226083159...","[(DK29627452, 0.8627519607543945), (DK40026088...","[(NO989847775, 0.9733341932296753), (DK4052688...","[(FI03983401, 0.7076908349990845), (DK27654436...","[(DK35664254, 0.7853683233261108), (SE55669512...","[(DK35243267, 1.0000005960464478), (FI22779844...","[(SE5590492210, 1.0000003576278687), (SE556812...","[(NO990638942, 0.9999995231628418), (DK2569885...","[(DK26933048, 0.9408202767372131), (DK34477396...","[(SE5591702112, 0.9148129224777222), (NO994967...","[(SE5566640263, 0.9430286884307861), (DK374395...","[(SE5566757497, 0.8797352313995361), (DK314292..."


In [19]:
out_df.to_csv('top_5_all.csv', header=True, index=True, mode='a')

In [20]:
new_df = pd.read_csv('top_5_all.csv', index_col=0)
new_df

,Computer vision.csv,Consulting.csv,Fintech.csv,Fish processing equipment.csv,Healthcare.csv,House builders.csv,Industrial vertical investor.csv,Innovative.csv,IoT.csv,IT freelance.csv,M&A advisors.csv,Manufacturers.csv,Online games.csv,Payments tech.csv,PE fund.csv,Procurement software.csv,Resource-efficiency.csv,Sustainability.csv,SaaS.csv,Wind turbine tech.csv
top_5,"[('NO914079845', 0.8554579019546509), ('SE5567...","[('DK36076011', 0.8274497985839844), ('DK12516...","[('SE5569622441', 0.904498279094696), ('FI0117...","[('FI18091246', 0.702485203742981), ('FI180912...","[('SE5567909428', 0.6710702180862427), ('SE556...","[('DK27920691', 0.9202667474746704), ('DK18411...","[('SE5563748309', 0.8689890503883362), ('FI168...","[('FI09899489', 0.7718906998634338), ('FI04135...","[('FI25904137', 1.0), ('NO990618976', 0.922608...","[('DK29627452', 0.8627519607543945), ('DK40026...","[('NO989847775', 0.9733341932296753), ('DK4052...","[('FI03983401', 0.7076908349990845), ('DK27654...","[('DK35664254', 0.7853683233261108), ('SE55669...","[('DK35243267', 1.0000005960464478), ('FI22779...","[('SE5590492210', 1.0000003576278687), ('SE556...","[('NO990638942', 0.9999995231628418), ('DK2569...","[('DK26933048', 0.9408202767372131), ('DK34477...","[('SE5591702112', 0.9148129224777222), ('NO994...","[('SE5566640263', 0.9430286884307861), ('DK374...","[('SE5566757497', 0.8797352313995361), ('DK314..."
NaN,Computer vision.csv,Consulting.csv,Fintech.csv,Fish processing equipment.csv,Healthcare.csv,House builders.csv,Industrial vertical investor.csv,Innovative.csv,IoT.csv,IT freelance.csv,M&A advisors.csv,Manufacturers.csv,Online games.csv,Payments tech.csv,PE fund.csv,Procurement software.csv,Resource-efficiency.csv,Sustainability.csv,SaaS.csv,Wind turbine tech.csv
top_5,"[('NO914079845', 0.8554579019546509), ('SE5567...","[('DK36076011', 0.8274497985839844), ('DK12516...","[('SE5569622441', 0.904498279094696), ('FI0117...","[('FI18091246', 0.702485203742981), ('FI180912...","[('SE5567909428', 0.6710702180862427), ('SE556...","[('DK27920691', 0.9202667474746704), ('DK18411...","[('SE5563748309', 0.8689890503883362), ('FI168...","[('FI09899489', 0.7718906998634338), ('FI04135...","[('FI25904137', 1.0), ('NO990618976', 0.922608...","[('DK29627452', 0.8627519607543945), ('DK40026...","[('NO989847775', 0.9733341932296753), ('DK4052...","[('FI03983401', 0.7076908349990845), ('DK27654...","[('DK35664254', 0.7853683233261108), ('SE55669...","[('DK35243267', 1.0000005960464478), ('FI22779...","[('SE5590492210', 1.0000003576278687), ('SE556...","[('NO990638942', 0.9999995231628418), ('DK2569...","[('DK26933048', 0.9408202767372131), ('DK34477...","[('SE5591702112', 0.9148129224777222), ('NO994...","[('SE5566640263', 0.9430286884307861), ('DK374...","[('SE5566757497', 0.8797352313995361), ('DK314..."
